Author: Alkistis Pourtsidou, ICG Portsmouth 

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html

To run this notebook you need to have CAMB and the CAMB python package installed. 
In order to install the CAMB python package on your computer follow the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [4]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [5]:
#Fiducial cosmological parameters
c=3e5
hubble=0.675
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9

nss = 0.9655
alphass = 0.0
betass = 0.0

gamma=0.545

kpivot = 0.05 
#kpivot = 0.1

In [6]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass, nrun=alphass, nrunrun=betass, pivot_scalar=kpivot)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [7]:
#calculate results for these parameters
results = camb.get_results(pars)

In [8]:
#Get matter power spectrum at z=0: P(k,z=0)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=1.0)
#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-5, maxkh=0.5, npoints = 800)

In [9]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [10]:
#Derivatives

def dlogPd_dns(kk):
    return np.log(kk/kpivot)

def dlogPd_dalphas(kk):
    return (1./2)*pow(np.log(kk/kpivot),2)

def dlogPd_dbetas(kk):
    return (1./6)*pow(np.log(kk/kpivot),3)

In [11]:
#Redshift bins
zlist = np.arange(0.7,2.05,0.1)
ztest = zlist[5]
Nzbins = len(zlist)

Area = 15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

#mean number density of galaxies
factor = pow(hubble,3)*1e-4
nbarlist = [17.5, 19, 18, 17, 15, 13, 12, 10, 8, 6, 4, 3.5, 2, 1]

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

Dzbin = 0.1


print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
ztest = 1.2
Number of redshift bins = 14


In [12]:
#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))

#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)

print rcom(ztest)

3881.21332089


In [13]:
#Define the growth function in LCDM
def get_growth(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

print get_growth(ztest)

0.901505284449


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return get_growth(zz)/(1+zz)
def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

print Dgz(ztest)

0.558232251833


In [15]:
#Construct Pgg(k,μ,z) 
def Pgg(kk,mu,zc):    
    return pow(bg+get_growth(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [16]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)


In [17]:
def kmin(zc):
    return 2*pi*pow(Vsur(zc),-1/3)

kNL = 0.1 #Mpc^{-1}    
    
def kmax(zc):
    return kNL*pow(1+zc,2/(2+nss)) #non-linear cutoff 

In [18]:
for zi in range(0,Nzbins):
    zc = zlist[zi]
    print zc, kmin(zc), kmax(zc)

0.7 0.00300391239432 0.143027630415
0.8 0.00285567190509 0.14864884108
0.9 0.00274250846588 0.154169231521
1.0 0.0026545118315 0.159595782335
1.1 0.00258515759285 0.164934667994
1.2 0.00252997099126 0.170191383714
1.3 0.00248578002456 0.17537084751
1.4 0.00245027417685 0.180477483171
1.5 0.00242173160884 0.185515288356
1.6 0.0023988439947 0.190487890971
1.7 0.00238060039977 0.195398596213
1.8 0.00236620816162 0.200250426112
1.9 0.00235503768985 0.205046153
2.0 0.00234658314854 0.209788328003


In [19]:
def Pshot(nbar):
    return 1/nbar

def Veff(kk,mu,zc):
    return Vsur(zc)*(Pgg(kk,mu,zc)/(Pgg(kk,mu,zc)+Pshot(nbar)))**2

In [20]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [21]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [22]:
mugrid = np.linspace(-1., 1., 513) 

In [23]:
#Fisher matrix parameters 
params = ["0:ns","1:alphas","2:betas"]

In [24]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [25]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [26]:
#%%time
#   Fisher matrix   # 

Npar = 2 #Npar = 3
#create array of zeros
s = (Npar,Npar)

Fishermat = np.zeros(s)

for zi in range(0,Nzbins):
    zc = zlist[zi]
    nbar = factor*nbarlist[zi]
    bg = biaslist[zi]
    kgrid = np.linspace(kmin(zc), kmax(zc), 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlogPd_dns(kk)
            elif i==1:  return dlogPd_dalphas(kk)
            elif i==2:  return dlogPd_dbetas(kk)
            else: print "out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlogPd_dns(kk)
                    elif j==1:  return dlogPd_dalphas(kk)
                    elif j==2:  return dlogPd_dbetas(kk)
                    else: print "index out of range" 
                Fishermat[i][j] += integrate2D(dF(K,MU),kgrid,mugrid)
            else: Fishermat[i,j] = Fishermat[j,i]

In [27]:
print Fishermat

[[ 2664131.9402093   1340721.35385563]
 [ 1340721.35385563   743896.81698823]]


In [28]:
print linalg.inv(Fishermat)

[[  4.03621557e-06  -7.27445028e-06]
 [ -7.27445028e-06   1.44549763e-05]]


In [29]:
#print marginalised uncertainties on (ns,alphas,betas)
for i in range(0,Npar):
    print np.sqrt(linalg.inv(Fishermat)[i,i])

0.00200903349255
0.00380197004973
